In [1]:
from modeling_florence2 import *
from configuration_florence2 import *
from transformers import AutoModelForCausalLM, AutoProcessor
import torch


/home/tkg5kq/.conda/envs/video2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tkg5kq/.conda/envs/video2/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
2025-02-21 06:35:51.410064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740137751.868743  398717 cuda_dnn.cc:8593] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740137751.994596  398717 cuda_blas.cc:14

In [2]:
# _MODELS = {
#     "BASE_FT": "microsoft/Florence-2-base-ft"
# }

# def available_models() -> List[str]:
#     """Returns the names of available CLIP models"""
#     return list(_MODELS.keys())
    
# def load(name: str, device: Union[str, torch.device] = "cuda" if torch.cuda.is_available() else "cpu"):
#     """Load the Florence Model

#     Parameters
#     ----------

#     Returns
#     -------
#     model : torch.nn.Module
#         The Florence Model
#     """
#     # download the model section (no download needed)
#     model_path = _MODELS[name]
    
#     # Build the model
#     auto_model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, revision='refs/pr/6').to(device)
#     processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True, revision='refs/pr/6')
#     state_dict = auto_model.state_dict()

#     config = auto_model.config
#     model = Florence2ForConditionalGeneration(
#         config
#     )

#     # convert_weights(model)
#     if pretrain:
#         model.load_state_dict(state_dict)

#     if str(device) == "cpu":
#         model.float()
    
#     return model2, model2.state_dict()


In [3]:
print(f'Available Models: {available_models()}')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
perceptor, vlm_state_dict, processor = load("BASE_FT", device)

Available Models: ['BASE_FT']


Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


DEBUG projection dim 768


In [4]:
# from modules.Visual_Prompt import visual_prompt
import yaml
from dotmap import DotMap
from modules.Visual_Prompt import *

class visual_prompt2(nn.Module):
    def __init__(self, sim_head, clip_state_dict, T):
        super().__init__()
        self.sim_header = sim_head
        self.T = T
        assert sim_head in ["meanP", "LSTM", "Transf", "Conv_1D", "Transf_cls"]

        if self.sim_header == "LSTM" or self.sim_header == "Transf" or self.sim_header == "Transf_cls" or self.sim_header == "Conv_1D":
            embed_dim = clip_state_dict["text_projection"].shape[1]

            context_length = clip_state_dict["positional_embedding"].shape[0]
            # vocab_size = clip_state_dict["token_embedding.weight"].shape[0]
            transformer_width = clip_state_dict["ln_final.weight"].shape[0]
            transformer_heads = transformer_width #// 64

            transformer_layers = len(
                set(k.split(".")[2] for k in clip_state_dict if k.startswith(f"transformer.resblocks")))

            self.frame_position_embeddings = nn.Embedding(context_length, embed_dim)
        if self.sim_header == "Transf" :
            self.transformer = TemporalTransformer(width=embed_dim, layers=6, heads=transformer_heads)
            print('layer=6')
        if self.sim_header == "LSTM":
            self.lstm_visual = nn.LSTM(input_size=embed_dim, hidden_size=embed_dim,
                                       batch_first=True, bidirectional=False, num_layers=1)

        self.apply(self.init_weights)

        if self.sim_header == "Transf_cls":
            self.transformer = TAggregate(clip_length=self.T, embed_dim=embed_dim, n_layers=6)

        if self.sim_header == 'Conv_1D' :
            self.shift = nn.Conv1d(embed_dim, embed_dim, 3, padding=1, groups=embed_dim, bias=False)
            weight = torch.zeros(embed_dim, 1, 3)
            weight[:embed_dim // 4, 0, 0] = 1.0
            weight[embed_dim // 4:embed_dim // 4 + embed_dim // 2, 0, 1] = 1.0
            weight[-embed_dim // 4:, 0, 2] = 1.0
            self.shift.weight = nn.Parameter(weight)

    def init_weights(self, module):
        """ Initialize the weights.
        """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            # Slightly different from the TF version which uses truncated_normal for initialization
            # cf https://github.com/pytorch/pytorch/pull/5617
            module.weight.data.normal_(mean=0.0, std=0.02)
        elif isinstance(module, LayerNorm):
            if 'beta' in dir(module) and 'gamma' in dir(module):
                module.beta.data.zero_()
                module.gamma.data.fill_(1.0)
            else:
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, x):
        b, t, c = x.size()
        x = x.contiguous()
        if self.sim_header == "meanP":
            pass
        elif self.sim_header == 'Conv_1D':
            x_original = x
            x = x.view(-1, c, t)
            x = self.shift(x.float())
            x = x.permute(0, 2, 1)
            x = x.type(x_original.dtype) + x_original

        elif self.sim_header == "Transf":
            x_original = x
            seq_length = t
            position_ids = torch.arange(seq_length, dtype=torch.long, device=x.device)
            position_ids = position_ids.unsqueeze(0).expand(x.size(0), -1)
            frame_position_embeddings = self.frame_position_embeddings(position_ids)
            x = x + frame_position_embeddings

            x = x.permute(1, 0, 2)  # NLD -> LND
            x = self.transformer(x)
            x = x.permute(1, 0, 2)  # LND -> NLD
            x = x.type(x_original.dtype) + x_original

        elif self.sim_header == "LSTM":
            x_original = x
            x, _ = self.lstm_visual(x.float())
            self.lstm_visual.flatten_parameters()
            x = torch.cat((x, x_original[:, x.size(1):, ...].contiguous()), dim=1)
            x = x.type(x_original.dtype) + x_original
        elif self.sim_header == "Transf_cls":
            x_original = x
            return self.transformer(x).type(x_original.dtype)

        else:
            raise ValueError('Unknown optimizer: {}'.format(self.sim_header))
        return x.mean(dim=1, keepdim=False)


In [6]:

with open('./configs/education/generic.yaml', 'r') as f:
    config = yaml.safe_load(f)
config = DotMap(config)

vlm_state_dict["text_projection"] = torch.empty((1, perceptor.config.text_config.d_model))
vlm_state_dict["positional_embedding"] = torch.empty((perceptor.config.text_config.vocab_size,))
vlm_state_dict["ln_final.weight"] = torch.empty((perceptor.config.text_config.encoder_attention_heads,))*64

# fusion_model = visual_prompt2(config.network.sim_header,vlm_state_dict,config.data.num_segments, perceptor.config.text_config)

In [11]:
print(perceptor.config.vision_config.dim_embed[-1])
print(perceptor.config.vision_config.projection_dim)


1024
768
